# Liquid Handler Program for GatorBio Assay
Generated from: GatorBio_Assay_Form.xlsm
Generated on: 2025-11-19 13:06:36
This notebook prepares assay and Max Plate (ProbeInfo) dilutions using pylabrobot.

## Setup Machine

In [ ]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources import Deck, Coordinate
from pylabrobot.liquid_handling import Strictness, set_strictness
from pylabrobot.resources.hamilton import STARDeck
import time

lh = LiquidHandler(backend=STARBackend(read_timeout=600), deck=STARDeck(core_grippers="1000uL-5mL-on-waste"))
await lh.setup(skip_iswap=True)
set_strictness(Strictness.STRICT)

## Set Variables
Change these values for your experiment.

In [ ]:
# Assay plate parameters
FINAL_VOLUME = 200  # µL transferred to assay 96-well plate
DILUTION_VOLUME = 300  # µL per dilution in deep well plate

# Max Plate parameters
MAX_PLATE_FINAL_VOLUME = 200  # µL transferred to Max Plate
MAX_PLATE_DILUTION_VOLUME = 300  # µL per dilution in Max Plate deep well

# Carrier Rail Assignment
PLATE_CARRIER_RAIL = 19
TUBE_CARRIER_RAIL = 35
TIP_CARRIER_RAIL = 7

# Mixing parameters
MIX_CYCLES = 4
CHANGE_TIPS_BETWEEN_DILUTIONS = False



## Stock Volume Requirements
Estimated minimum volumes to load in each 50 mL tube (add extra for dead volume):
- `stock_buffer`: 28.30 mL (≈ 28301 µL)
- `stock_neutralization`: 1.60 mL (≈ 1600 µL)
- `stock_regeneration`: 1.60 mL (≈ 1600 µL)

Probe wells buffer usage is included in the `stock_buffer` total.


## Sample Stock Placement
- `3-12` → `dilution_plate['A1']` (0.40 mL) (includes reserve for next dilution)
- `4-2` → `dilution_plate['B1']` (0.40 mL) (includes reserve for next dilution)
- `4-3` → `dilution_plate['C1']` (0.40 mL) (includes reserve for next dilution)
- `4-4` → `dilution_plate['D1']` (0.40 mL) (includes reserve for next dilution)
- `6-1` → `dilution_plate['E1']` (0.40 mL) (includes reserve for next dilution)
- `6-2` → `dilution_plate['F1']` (0.40 mL) (includes reserve for next dilution)
- `EPO-R` → `dilution_plate['C5']` (2.40 mL)
- `WT` → `dilution_plate['E5']` (0.40 mL) (includes reserve for next dilution)
- `6-3` → `dilution_plate['G1']` (0.40 mL) (includes reserve for next dilution)
- `6-4` → `dilution_plate['H1']` (0.40 mL) (includes reserve for next dilution)
- `6-5` → `dilution_plate['A5']` (0.40 mL) (includes reserve for next dilution)
- `6-6` → `dilution_plate['B5']` (0.40 mL) (includes reserve for next dilution)
- `Probe` → `dilution_plate['D5']` (2.40 mL)


In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cor_96_wellplate_360ul_Fb,
    nest_96_wellplate_2mL_Vb,
    hamilton_96_tiprack_1000uL,
    Tube_CAR_32_A00,
    hamilton_tube_carrier_12_b00,
    Cor_Falcon_tube_50mL_Vb,
)
from pylabrobot.liquid_handling.standard import Mix

lh.deck.get_resource("trash_core96").location = Coordinate(-260, 106, 216.4)

# Tips
tip_car = TIP_CAR_480_A00(name="tip_carrier")
tip_car[0] = hamilton_96_tiprack_1000uL(name="main_tips")
tip_car[1] = hamilton_96_tiprack_1000uL(name="spare_tips")
lh.deck.assign_child_resource(tip_car, rails=TIP_CARRIER_RAIL)

TIP_WELL_ORDER = [
    'A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1',
    'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2',
    'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3',
    'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4',
    'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5',
    'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6',
    'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7',
    'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8',
    'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9',
    'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10',
    'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11',
    'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12',
]
TIP_POSITIONS = [("main_tips", pos) for pos in TIP_WELL_ORDER] + [("spare_tips", pos) for pos in TIP_WELL_ORDER]
tip_position_index = 0

# Plates
dilution_plate_names = ['dilution_plate']
plt_car = PLT_CAR_L5AC_A00(name="plate_carrier")
plt_car[0] = Cor_96_wellplate_360ul_Fb(name="final_plate")  # Assay plate
plt_car[1] = Cor_96_wellplate_360ul_Fb(name="max_plate_final")  # Max Plate
plt_car[2] = nest_96_wellplate_2mL_Vb(name=dilution_plate_names[0])  # Shared dilutions
if len(dilution_plate_names) > 1:
    plt_car[3] = nest_96_wellplate_2mL_Vb(name=dilution_plate_names[1])  # Additional dilutions
lh.deck.assign_child_resource(plt_car, rails=PLATE_CARRIER_RAIL)

# 50 mL stock tubes
stock_resources = ['stock_buffer', 'stock_neutralization', 'stock_regeneration']
tube_car = hamilton_tube_carrier_12_b00(name="tube_carrier")
for i, resource_name in enumerate(stock_resources):
    if i >= 12:
        print(f"Warning: Not enough tube positions for {resource_name}")
        continue
    tube_car[11 - i] = Cor_Falcon_tube_50mL_Vb(name=resource_name)
lh.deck.assign_child_resource(tube_car, rails=TUBE_CARRIER_RAIL)

lh.summary()

## Helper Functions

In [ ]:
def next_tip_positions(count):
    global tip_position_index
    if tip_position_index + count > len(TIP_POSITIONS):
        raise RuntimeError('Not enough tips available for this run.')
    positions = TIP_POSITIONS[tip_position_index:tip_position_index + count]
    tip_position_index += count
    resources = []
    for rack_name, position in positions:
        resources.append(lh.deck.get_resource(rack_name)[position][0])
    return resources

def group_wells_by_column(wells):
    columns = {}
    for well in wells:
        column = ''.join([ch for ch in well if ch.isdigit()])
        columns.setdefault(column, []).append(well)
    for column in columns:
        columns[column].sort()
    return columns

async def multi_channel_transfer_from_tube(tube_resource_name, plate_name, column_transfers, keep_tips=False):
    # column_transfers: List[[column_number, volume_list_8]]
    tube = lh.deck.get_resource(tube_resource_name)
    plate = lh.deck.get_resource(plate_name)
    MAX_TIP_VOLUME = 1000
    channel_set = set()
    per_channel_total = [0]*8
    for column_number, volumes in column_transfers:
        channel_indices = [idx for idx, vol in enumerate(volumes) if vol and vol > 0]
        channel_set.update(channel_indices)
        for idx in channel_indices:
            per_channel_total[idx] += volumes[idx]
    # Check tip presence from machine
    tip_presence = await lh.backend.request_tip_presence()
    channels_with_tips = [i for i, has_tip in enumerate(tip_presence) if has_tip == 1]
    # Check if we need to discard tips (if required channels don't match existing tips)
    if not set(channel_set).issubset(set(channels_with_tips)) and not keep_tips:
        await lh.discard_tips()
        tip_resources = next_tip_positions(len(channel_set))
        await lh.pick_up_tips(tip_resources, use_channels=list(channel_set))
    print(f'[DEBUG] multi_channel_transfer_from_tube: channels={channel_set}')
    print(f'[DEBUG] Total volumes to aspirate per channel (row_idx -> volume): {per_channel_total}')
    for idx, total in enumerate(per_channel_total):
        if total > MAX_TIP_VOLUME:
            raise ValueError(f'Requested volume {total} µL exceeds 1000 µL limit for tip channel {idx}.')
    for idx in channel_set:
        print(f'[DEBUG] Aspirating {per_channel_total[idx]:.1f} µL into channel {idx}')
        await lh.aspirate(
            [tube],
            vols=[per_channel_total[idx]],
            use_channels=[idx],
        )
    # Check volume in tips after aspiration
    print('[DEBUG] Checking volume in tips after aspiration:')
    for channel in channel_set:
        vol_in_tip = await lh.backend.request_volume_in_tip(channel)
        expected_vol = per_channel_total[channel]
        print(f'  Channel {channel}: {vol_in_tip:.1f} µL (expected: {expected_vol:.1f} µL)')
    for column_number, volumes in column_transfers:
        channels_to_use = []
        volumes_filtered = []
        for idx, volume in enumerate(volumes):
            if volume and volume > 0:
                channels_to_use.append(idx)
                volumes_filtered.append(volume)
        target_positions = [f"{chr(ord('A') + channel)}{column_number}" for channel in channels_to_use]
        targets_all = [plate[pos][0] for pos in target_positions]
        await lh.dispense(
            targets_all,
            vols=volumes_filtered,
            use_channels= channels_to_use,
            offsets=[Coordinate(0, 0, 5)] * len(channels_to_use),
        )

async def multi_channel_serial_dilution(source_plate_name, target_plate_name, source_column, target_column, transfer_volumes, keep_tips=False):
    channel_indices = [idx for idx, vol in enumerate(transfer_volumes) if vol and vol > 0]
    source_plate = lh.deck.get_resource(source_plate_name)
    target_plate = lh.deck.get_resource(target_plate_name)
    source_positions = [f"{chr(ord('A') + idx)}{source_column}" for idx in channel_indices]
    target_positions = [f"{chr(ord('A') + idx)}{target_column}" for idx in channel_indices]
    # Check tip presence from machine
    tip_presence = await lh.backend.request_tip_presence()
    channels_with_tips = [i for i, has_tip in enumerate(tip_presence) if has_tip == 1]
    # Check if we need to discard tips (if required channels don't match existing tips)
    if not set(channel_indices).issubset(set(channels_with_tips)) and not keep_tips:
        await lh.discard_tips()
        tip_resources = next_tip_positions(len(channel_indices))
        await lh.pick_up_tips(tip_resources, use_channels=list(channel_indices))
    source_containers = [source_plate[pos][0] for pos in source_positions]
    transfer_list = [transfer_volumes[idx] for idx in channel_indices]
    await lh.aspirate(source_containers, vols=transfer_list, use_channels=channel_indices)
    target_containers = [target_plate[pos][0] for pos in target_positions]
    mix_args = None
    if MIX_CYCLES and MIX_CYCLES > 0:
        mix_args = [Mix(volume=transfer_volumes[idx], repetitions=MIX_CYCLES) for idx in channel_indices]
    await lh.dispense(target_containers, vols=transfer_list, use_channels=channel_indices, mix=mix_args)

async def transfer_to_final_plate(target_plate_name, entries, volume):
    """Transfer liquid from source plate to target plate"""
    ordered = sorted(entries)
    if not ordered:
        return
    target_plate = lh.deck.get_resource(target_plate_name)
    source_containers = [lh.deck.get_resource(source_plate_name)[source_well][0] for source_plate_name, source_well, _ in ordered]
    target_containers = [target_plate[target_well][0] for _, _, target_well in ordered]
    tips = next_tip_positions(len(ordered))
    await lh.pick_up_tips(tips)
    if len(set([x[1] for x in ordered])) != len(ordered):
        for i, source_well in enumerate(source_containers):
            await lh.aspirate([source_well], vols=[volume], use_channels=[i], blow_out_air_volume=[10])
    else:
        await lh.aspirate(
            source_containers,
            vols=[volume] * len(ordered),
            use_channels=list(range(len(ordered))),
            blow_out_air_volume=[10] * len(ordered),
        )
    await lh.dispense(
        target_containers,
        vols=[volume] * len(ordered),
        use_channels=list(range(len(ordered))),
        blow_out_air_volume=[15] * len(ordered),
        offsets=[Coordinate(0, 0, 5)] * len(ordered),
    )
    await lh.discard_tips()


## Execute Liquid Handling

In [ ]:
print("Starting liquid handling...")

print("Starting sample stock transfers...")
# Ensure 3-12 sample stock (399.9 µL) is pre-loaded into dilution_plate well A1
# Ensure 4-2 sample stock (399.9 µL) is pre-loaded into dilution_plate well B1
# Ensure 4-3 sample stock (399.9 µL) is pre-loaded into dilution_plate well C1
# Ensure 4-4 sample stock (399.9 µL) is pre-loaded into dilution_plate well D1
# Ensure 6-1 sample stock (399.9 µL) is pre-loaded into dilution_plate well E1
# Ensure 6-2 sample stock (399.9 µL) is pre-loaded into dilution_plate well F1
# Ensure EPO-R load stock (2400.0 µL) is pre-loaded into dilution_plate well C5
# Ensure WT sample stock (399.9 µL) is pre-loaded into dilution_plate well E5
# Ensure 6-3 sample stock (399.9 µL) is pre-loaded into dilution_plate well G1
# Ensure 6-4 sample stock (399.9 µL) is pre-loaded into dilution_plate well H1
# Ensure 6-5 sample stock (399.9 µL) is pre-loaded into dilution_plate well A5
# Ensure 6-6 sample stock (399.9 µL) is pre-loaded into dilution_plate well B5
# Ensure Probe buffer (2400.0 µL) is pre-loaded into dilution_plate well D5

print("Prefilling buffer into dilution plate columns...")
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', [[2, [300.1, 300.1, 300.1, 300.1, 300.1, 300.1, 300.1, 300.1]], [3, [300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0]], [4, [300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0]]], keep_tips=True)
await multi_channel_transfer_from_tube('stock_buffer', 'dilution_plate', [[6, [300.1, 300.1, 0.0, 0.0, 300.1, 0.0, 0.0, 0.0]], [7, [300.0, 300.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0]], [8, [300.0, 300.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0]]], keep_tips=True)

print("Loading stock_buffer into final_plate...")
await multi_channel_transfer_from_tube('stock_buffer', 'final_plate', [[1, [200, 200, 200, 200, 200, 200, 200, 200]], [3, [200, 200, 200, 200, 200, 200, 200, 200]], [4, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]], [5, [200, 200, 200, 200, 200, 200, 200, 200]], [8, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]]], keep_tips=True)
print("stock_buffer: loaded 26 wells on final_plate")
print("Loading stock_buffer into final_plate...")
await multi_channel_transfer_from_tube('stock_buffer', 'final_plate', [[9, [200, 200, 200, 200, 200, 200, 200, 200]], [10, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]], [11, [200, 200, 200, 200, 200, 200, 200, 200]], [12, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]]], keep_tips=True)
print("stock_buffer: loaded 18 wells on final_plate")
print("Loading stock_buffer into max_plate_final...")
await multi_channel_transfer_from_tube('stock_buffer', 'max_plate_final', [[1, [200, 200, 200, 200, 200, 200, 200, 200]], [2, [200, 200, 200, 200, 200, 200, 200, 200]], [3, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]], [4, [200, 200, 200, 200, 200, 200, 200, 200]], [5, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200]]], keep_tips=True)
print("stock_buffer: loaded 26 wells on max_plate_final")
print("Loading stock_buffer into max_plate_final...")
await multi_channel_transfer_from_tube('stock_buffer', 'max_plate_final', [[6, [200, 200, 200, 200, 200, 200, 200, 200]], [7, [0.0, 0.0, 200, 200, 200, 200, 200, 200]], [8, [200, 200, 200, 200, 200, 200, 200, 200]]], keep_tips=False)
print("stock_buffer: loaded 22 wells on max_plate_final")

print("Loading stock_neutralization into final_plate...")
await multi_channel_transfer_from_tube('stock_neutralization', 'final_plate', [[7, [200, 200, 200, 200, 200, 200, 200, 200]]], keep_tips=False)
print("stock_neutralization: loaded 8 wells on final_plate")

print("Loading stock_regeneration into final_plate...")
await multi_channel_transfer_from_tube('stock_regeneration', 'final_plate', [[6, [200, 200, 200, 200, 200, 200, 200, 200]]], keep_tips=False)
print("stock_regeneration: loaded 8 wells on final_plate")

print("Starting serial dilutions...")
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 1, 2, [99.9, 99.9, 99.9, 99.9, 99.9, 99.9, 99.9, 99.9], keep_tips=False)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 2, 3, [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], keep_tips=True)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 3, 4, [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], keep_tips=True)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 5, 6, [99.9, 99.9, 0.0, 0.0, 99.9, 0.0, 0.0, 0.0], keep_tips=False)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 6, 7, [100.0, 100.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0], keep_tips=True)
await multi_channel_serial_dilution('dilution_plate', 'dilution_plate', 7, 8, [100.0, 100.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0], keep_tips=True)
await lh.discard_tips()

print("Transferring assay dilutions to final plate...")
await transfer_to_final_plate('final_plate', [('dilution_plate', 'A4', 'A8'), ('dilution_plate', 'B1', 'B8'), ('dilution_plate', 'B2', 'C8'), ('dilution_plate', 'B3', 'D8'), ('dilution_plate', 'A1', 'E4'), ('dilution_plate', 'B4', 'E8'), ('dilution_plate', 'A2', 'F4'), ('dilution_plate', 'A3', 'G4')], FINAL_VOLUME)
await transfer_to_final_plate('final_plate', [('dilution_plate', 'C3', 'A10'), ('dilution_plate', 'C4', 'B10'), ('dilution_plate', 'D1', 'C10'), ('dilution_plate', 'D2', 'D10'), ('dilution_plate', 'D3', 'E10'), ('dilution_plate', 'C1', 'F8'), ('dilution_plate', 'D4', 'F10'), ('dilution_plate', 'C2', 'G8')], FINAL_VOLUME)
await transfer_to_final_plate('final_plate', [('dilution_plate', 'E2', 'A12'), ('dilution_plate', 'E3', 'B12'), ('dilution_plate', 'E4', 'C12'), ('dilution_plate', 'F1', 'D12'), ('dilution_plate', 'F2', 'E12'), ('dilution_plate', 'F3', 'F12'), ('dilution_plate', 'E1', 'G10'), ('dilution_plate', 'F4', 'G12')], FINAL_VOLUME)
await transfer_to_final_plate('final_plate', [('dilution_plate', 'C5', 'A2'), ('dilution_plate', 'C5', 'B2'), ('dilution_plate', 'C5', 'C2'), ('dilution_plate', 'C5', 'D2'), ('dilution_plate', 'C5', 'E2'), ('dilution_plate', 'C5', 'F2'), ('dilution_plate', 'C5', 'G2'), ('dilution_plate', 'C5', 'H2')], FINAL_VOLUME)
await transfer_to_final_plate('final_plate', [('dilution_plate', 'E5', 'A4'), ('dilution_plate', 'E6', 'B4'), ('dilution_plate', 'E7', 'C4'), ('dilution_plate', 'E8', 'D4')], FINAL_VOLUME)


print("Transferring Max Plate dilutions to final plate...")
await transfer_to_final_plate('max_plate_final', [('dilution_plate', 'G1', 'A3'), ('dilution_plate', 'H4', 'A5'), ('dilution_plate', 'G2', 'B3'), ('dilution_plate', 'G3', 'C3'), ('dilution_plate', 'G4', 'D3'), ('dilution_plate', 'H1', 'E3'), ('dilution_plate', 'H2', 'F3'), ('dilution_plate', 'H3', 'G3')], MAX_PLATE_FINAL_VOLUME)
await transfer_to_final_plate('max_plate_final', [('dilution_plate', 'B7', 'A7'), ('dilution_plate', 'A5', 'B5'), ('dilution_plate', 'B8', 'B7'), ('dilution_plate', 'A6', 'C5'), ('dilution_plate', 'A7', 'D5'), ('dilution_plate', 'A8', 'E5'), ('dilution_plate', 'B5', 'F5'), ('dilution_plate', 'B6', 'G5')], MAX_PLATE_FINAL_VOLUME)
await transfer_to_final_plate('max_plate_final', [('dilution_plate', 'D5', 'A1'), ('dilution_plate', 'D5', 'B1'), ('dilution_plate', 'D5', 'C1'), ('dilution_plate', 'D5', 'D1'), ('dilution_plate', 'D5', 'E1'), ('dilution_plate', 'D5', 'F1'), ('dilution_plate', 'D5', 'G1'), ('dilution_plate', 'D5', 'H1')], MAX_PLATE_FINAL_VOLUME)


print("Liquid handling complete!")